# Ever wanted to use a real quantum computer?

Good news! Thanks to [*IBM Quantum*](https://quantum-computing.ibm.com/), anyone can access a handful of small QPUs over the cloud, free of charge. In this notebook, we'll use [*Covalent*](http://covalent.xyz) with [*Pennylane*](https://pennylane.ai/) and  *IBM Quantum* hardware to train an SVM classifier with a real quantum kernel circuit.

## Preliminaries

Visit [quantum-computing.ibm.com](https://quantum-computing.ibm.com/) and sign up for a (free) IBMid.

<img src="http://drive.google.com/uc?export=view&id=1aK2S4UXwjJ8N07Goq6E3sl40H3l7hnMf">

Next, log in and copy the API token (a string of characters) on the landing page. This will grant programmatic access to IBM's QPUs.

<img src="http://drive.google.com/uc?export=view&id=1LdYOTq3K4apPXWYTjJYoUsiZ5A11Z8Uy">

Set the `IBM_Q_API_TOKEN` variable in the [Settings](#settings) cell below.

*Remember, you should never publish any code that reveals the API token - **including this notebook, once it's completed.***

A few more things to note:

* QPUs are in high demand. Wait times on the order of hours per task are not unlikely.
* Once you have an IBMid, you can monitor and delete/cancel any submitted jobs using the same portal where you obtained the API token.

<img src="http://drive.google.com/uc?export=view&id=1LYkXBLKkKsvGZJf3syj1ML905gqfGgLr">

That's it! We're good to go.

## *Challenge*

As is, this notebook runs a toy SVM classification task using parallel calls to the quantum kernel circuit running on a local simulator (Pennylane's `'lightning.qubit'` device). Follow the [Pennylane documentation](https://docs.pennylane.ai/projects/qiskit/en/latest/devices/ibmq.html) and modify the [QPU sub-task](#qpu-sub-task) to use an *IBM Quantum* hardware backend.
___________________________________

# An SVM classifier with a quantum kernel... on real quantum hardware!

## Install some packages

| package | Description |
| ------- | ----------- |
|`scikit-learn`| classical SVM implementation|
|`pennylane` | quantum programming framework and simulator|
|`matplotlib`| plotting library|


In [ ]:
# !python -m pip install scikit-learn
# !python -m pip install pennylane
# !python -m pip install pennylane-qiskit
# !python -m pip install matplotlib
# !python -m pip install covalent

## Imports

In [ ]:
import random
from math import pi as PI

import covalent as ct
import matplotlib.pyplot as plt
import numpy as np
import pennylane as qml
from pennylane import numpy as np
from sklearn.datasets import load_wine
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

## Data for our example

For compatibility with freely-available quantum hardware, we reduce the 'wine recognition' dataset to 6 training and 2 testing samples. Additionally, only two cultivators are considered, 'R' and 'G'.

This data reproduced by the `get_data` electron (with `RANDOM_SEED = 2022`) in the [Local sub tasks](#local-sub-tasks) section.

**We pursue the same goal in this toy example:** recognize the cultivator based on the wine's *hue* and *colour intensity*.

<img src="http://drive.google.com/uc?export=view&id=10n4ASvh2SXSijqv-PozPtvI1u7AQbgDi">

# Settings

paste the API token below

In [ ]:
IBM_Q_API_TOKEN = "paste-your-API-token-here"

no need to modify any of the following settings

In [1]:
N_TEST = 2
N_TRAIN = 6
N_TOT = N_TEST + N_TRAIN

RANDOM_SEED = 2022

IBM_HARDWARE_BACKENDS = [
    'ibmq_lima',
    'ibmq_belem',
    'ibmq_quito',
    'ibmq_manila',
    'ibm_nairobi',
    'ibm_oslo'
]

## QPU sub-task

The electron defined in the cell below will be executed on an IBM QPU.

For instructions, see the [Pennylane documentation](https://docs.pennylane.ai/projects/qiskit/en/latest/devices/ibmq.html).

In [ ]:
# a projector onto the state |00> -- i.e. the matrix |00><00|
OPERATOR = qml.Hermitian(np.array([[1., 0., 0., 0.],
                                   [0., 0., 0., 0.],
                                   [0., 0., 0., 0.],
                                   [0., 0., 0., 0.]]), wires=range(2))

# ------------------------------------------------------------------------------
# PROMPT 
#
# The code below runs the quantum kernel computation on a local simulator.
# Modify this code to instead use an IBM Quantum device backend.
#
# ------------------------------------------------------------------------------
dev = qml.device("lightning.qubit", wires=2, shots=1000)  # local simulator

@ct.electron
@qml.qnode(dev)
def qkernel_circuit(x1, x2):
    """evaluate the quantum kernel circuit on a pair of data points"""

    qml.AngleEmbedding(features=x1, wires=range(2))
    qml.adjoint(qml.AngleEmbedding)(features=x2, wires=range(2))

    # estimate expectation value of `OPERATOR` over 1000 measurement samples
    return qml.expval(OPERATOR)


## Local sub-tasks

These electrons will be executed locally.

In [ ]:
@ct.electron
def get_data():
    """load reduced 'wine recognition' data and select desired columns"""
    features, labels = load_wine(return_X_y=True)
    X = features[:, 9:11]  # pick out columns "colour intensity" and "hue"
    y = labels
    
    # neglect third class
    idx = (y < 2)
    y = y[idx]
    X = X[idx]

    scaler = MinMaxScaler((0, 2 * PI))
    scaler.fit(X)

    return scaler.transform(X), y


@ct.electron
def partition_data(X, y):
    """perform a train/test split"""
    _, X0, _, y0 = train_test_split(X, y,
                                    test_size=N_TOT / len(X),
                                    random_state=RANDOM_SEED,
                                    stratify=y)

    X_train, X_test, y_train, y_test = train_test_split(X0, y0,
                                                        test_size=N_TEST / N_TOT,
                                                        random_state=RANDOM_SEED,
                                                        stratify=y0)
    return X_train, X_test, y_train, y_test


@ct.electron
def construct_K_matrix(K_values, X1, X2):
    """return a blank kernel matrix for the array of data points"""
    n1 = len(X1)
    n2 = len(X2)
    return np.array(K_values).reshape((n1, n2))


@ct.electron
def train_qsvm(K_matrix_train, y_train):
    """train the support-vector classifier (SVC)"""
    svc = SVC()
    svc.fit(K_matrix_train, y_train)
    return svc


@ct.electron
def get_metrics(trained_svc, K_matrix_test, y_test):
    """compute classifier metrics"""
    y_pred = trained_svc.predict(K_matrix_test)
    return y_pred, confusion_matrix(y_test, y_pred)

## Workflow

In [ ]:
@ct.lattice
def workflow():
    """get the data, partition it, and train SVC on it"""
    
    # load data and partition it
    X, y = get_data()
    X_train, X_test, y_train, y_test = partition_data(X, y)
    
    # individually compute kernel matrix elements on the QPU
    K_train_values = []
    for i in range(N_TRAIN):
        for j in range(N_TRAIN):
            kernel_value = qkernel_circuit(X_train[i], X_train[j])
            K_train_values.append(kernel_value)

    K_test_values = []
    for i in range(N_TEST):
        for j in range(N_TRAIN):
            kernel_value = qkernel_circuit(X_test[i], X_train[j])
            K_test_values.append(kernel_value)
    
    # construct kernel matrices from results
    K_matrix_train = construct_K_matrix(K_train_values, X_train, X_train)
    K_matrix_test = construct_K_matrix(K_test_values, X_test, X_train)
    
    # train the classifier
    trained_svc = train_qsvm(K_matrix_train, y_train)
    
    # get predictions and confusion matrix
    y_pred, M_conf = get_metrics(trained_svc, K_matrix_test, y_test)

    return (X_train, X_test, y_train, y_test), y_pred, M_conf

## Dispatch

In [ ]:
dispatch_id = ct.dispatch(workflow)()
result = ct.get_result(dispatch_id, wait=True)

# unpack result
data, y_pred, M_conf = result.result

## Processing and displaying results

In [ ]:
X_train, X_test, y_train, y_test = data

names = ("cultivator 'R'",
         "cultivator 'G'")

colors = ("r", "g")

for i in range(2):
    plt.scatter(X_train[:, 0][y_train == i], X_train[:, 1]
                [y_train == i], c=colors[i], s=100)

idx = np.equal(y_pred, y_test)

for i in range(2):
    # correct
    plt.scatter(X_test[:, 0][np.logical_and(y_test == i, idx)],
                X_test[:, 1][np.logical_and(y_test == i, idx)],
                c='w', marker='^', s=200, edgecolors=colors[i], label=names[i])

    # incorrect
    plt.scatter(X_test[:, 0][np.logical_and(y_test == i, ~idx)],
                X_test[:, 1][np.logical_and(y_test == i, ~idx)],
                s=200, c=colors[i], marker='x')


plt.title("wine colour metrics", fontsize=16)
plt.xlabel("intensity (scaled)", fontsize=13)
plt.ylabel("hue (scaled)", fontsize=13)
plt.grid()
plt.legend(fontsize=15)
plt.show()


ConfusionMatrixDisplay(M_conf, display_labels=["ctvr. R",
                                               "ctvr. G"]).plot()